# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [65]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [66]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [67]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [68]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [69]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [70]:
try:
    session.execute("""
    create keyspace if not exists udacity
    with replication =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [71]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [72]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS song_info_by_session "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, primary key(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [73]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_info_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + " values (%s, %s, %s, %s, %s) "
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

### To answer the first question, we have created a table *song_info_by_session* which consists of 5 columns:

 - *sessionId*: stores the session id, data type: int
 - *itemInSession*: stores the item number in session, data type: int
 - *artist*: stores the artist name, data type: text
 - *song*: stores the song title, data type: text
 - *length*: stores the song's length, data type: float
    
The partition key is *sessionId* so that all records from the same session are stored in one partition, and the clustering key is *itemInSession* that is used to sort the records in a session.

In [74]:
query = "select artist, song, length from song_info_by_session WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [35]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS song_info_by_user "
query = query + "(userid int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, primary key((userid, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [36]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_info_by_user (userid, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " values (%s, %s, %s, %s, %s, %s, %s) "
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### To answer the second question, we have creates a table *song_info_by_user* which consists of the following columns:

- *userid*: stores the user id, data type: int
- *sessionId*: stores the session id, data type: int
- *itemInSession*: stores the item number in session, data type: int
- *artist*: stores the artist name, data type: text
- *song*: stores the song title, data type: text
- *firstName*: stores the first name of user, data type: text
- *lastName*: stores the last name of, data type: text

The partition keys are *userid* and *sessionId*. We use both *userid* and *sessionId* as primary keys so that sessions from the same user are stored in the same nodes.

The clustering key is *itemInSession* as we need songs sorted by item in section.

In [37]:
query = "select artist, song, firstName, lastName from song_info_by_user where userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [55]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS user_info_by_song "
query = query + "(song text, userid int, firstName text, lastName text, primary key(song, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [56]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into user_info_by_song (song, userid, firstName, lastName)"
        query = query + " values (%s, %s, %s, %s) "
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

### To answer the third question, we have created a table *user_info_by_song* which consists of the following columns:

- *song*: stores the song title, data type: text
- *userid*: stores the user id, data type: int
- *firstName*: stores the first name of user, data type: text
- *lastName*: stores the last name of, data type: text

The partition key is *song*, and the clustering key is *userid*. We have included *userid* in primary key to ensure the rows unique.

In [57]:
query = "select song, firstName, lastName from user_info_by_song where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song, row.firstname, row.lastname)

All Hands Against His Own Jacqueline Lynch
All Hands Against His Own Tegan Levine
All Hands Against His Own Sara Johnson


### Drop the tables before closing out the sessions

In [61]:
query = "drop table if exists song_info_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [62]:
query = "drop table if exists song_info_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [63]:
query = "drop table if exists user_info_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [64]:
session.shutdown()
cluster.shutdown()